# 4.利用层级索引实现OLAP
Pandas提供层级索引（hierarchical indexing，也称为多级索引，multi-indexing）来描述数据维度中的层级结构。在这一节中，我们将介绍MultiIndex对象的创建方法，以及利用它来实现切片、切块、统计等联机数据分析（OLAP）操作。

In [ ]:
import pandas as pd
import numpy as np

## 4.1 笨办法
下面以美国各州在不同年份的人口数据统计为案例，首先，如果用单级索引，那么，索引的键值应该由“州”和“年份”两类信息组成。

In [62]:
index = [('California', 2000), ('California', 2010), ('New York', 2000), ('New York', 2010), ('Texas', 2000), ('Texas', 2010)]
population = [33871648, 37253956, 18976457, 19378102, 20851820, 25145561]
pop = pd.Series(population, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

利用元组构成的索引可以查询单年的数据：

In [50]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

但是不便于做统计，比如选择所有2000年的数据：

In [51]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

##  4.2 多级索引
比如，我们可以直接用上面的元组列表来创建多级索引：

In [63]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

<p>MultiIndex里由一个levels属性表示索引的等级，将州名和年份作为每个数据点的不同标签。</p>
<p>再将pop的索引重置为多级索引</p>

In [64]:
pop=pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

用多级索引查询2000年的数据：

In [ ]:
pop[:, 2000]

In [ ]:
pop['California']

多级索引可以用DataFrame来代替，其实，Pandas已经提供了对应的功能。用Series的unstack()方法可以快速将一个多级索引的Series转换成普通索引的DataFrame。

In [46]:
pop_df = pop.unstack()
pop_df

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


也可以用 stack() 方法实现相反的操作

In [47]:
pop_df.stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

虽然可以用DataFrame代替多级索引，但是实际使用中，多级索引可以更清晰地显示数据维度之间的层级结构，索引每增加一维，数据集能表示的维度就能够增加一维。（见4.3.4 多级列索引）

## 4.3 创建带多级索引的Series和DataFrame
### 4.3.1 隐式地创建
在创建Series和DataFrame对象时，将index参数设置为至少二维的多维数组，Pandas会自动将其创建为多级索引。

In [ ]:
df = pd.DataFrame(np.random.rand(4,2),
                 index=[['a','a','b','b'],[1,2,1,2]],
                 columns=['data1','data2'])
df

将元组作为字典传递给Pandas，也会自动创建多级索引，此方法比较适合用于Series的创建。

In [ ]:
data = {('California', 2000):33871648, ('California', 2010):37253956, ('New York', 2000):18976457, ('New York', 2010):19378102, ('Texas', 2000):20851820, ('Texas', 2010):25145561}
pd.Series(data)

### 4.3.2 显示地创建
隐式创建方法中，需要列出所有索引的全部取值，显然不适合用于大规模数据。

Pandas提供了灵活的、显式地MultiIndex创建方法，从多维数组、元组列表、两个索引的笛卡尔乘积、键值和标签值列表中都可以创建多级索引。

然后，可以在创建Series和DataFrame时利用index参数指定多级索引对象，或者通过reindex方法更新Series或DataFrame的索引。

（1）从数组中创建

In [ ]:
pd.MultiIndex.from_arrays([['California','California','New York','New York','Texas','Texas'], [2000,2010,2000,2010,2000,2010]])

（2）从元组中创建

In [ ]:
pd.MultiIndex.from_tuples([('California', 2000), ('California', 2010), ('New York', 2000), ('New York', 2010), ('Texas', 2000), ('Texas', 2010)])

<b>（3）从两个列表的笛卡尔乘积创建</b>

这种方法的优点是只需要列出一次每级索引的内容，没有重复；

In [ ]:
index=pd.MultiIndex.from_product([['California','New York','Texas'],[2000,2010]])
pop.reindex(index)

缺点是会产生所有可能组合，如果有的值没有，就会产生空值：

In [ ]:
index=pd.MultiIndex.from_product([['California','New York','Texas'],[2000,2010,2020]])
pop.reindex(index)

而用前面方法创建的MultiIndex则允许某些下级索引只存在一些一级索引值中，比如：

In [ ]:
index=pd.MultiIndex.from_tuples([('California', 2000), ('California', 2010), ('California', 2020), ('New York', 2000), ('New York', 2010), ('Texas', 2000), ('Texas', 2010)])
pop.reindex(index)

（4）通过指定levels和labels创建

In [ ]:
pd.MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
             labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

### 4.3.3 多级索引的等级名称

给MultiIndex的等级加上名称可以增加可读性，也可以为读取操作提供便利。可以在构造器中通过 names 参数设置，也可以在创建之后通过 names 属性来指定。

In [65]:
pop.index.names=['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### 4.3.4 多级列索引
多级索引既可以为行设置，也可以为列设置。下面以一个以医学数据表为例进行演示。

In [19]:
index = pd.MultiIndex.from_product([[2013,2014], [1,2]], names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']], names=['subject','type'])

data = np.round(np.random.randn(4,6), 1)
data[:,::2] *= 10
data += 37

health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      31.0  38.3  34.0  37.4  35.0  39.5
     2      45.0  36.2  42.0  35.2  28.0  35.4
2014 1      40.0  37.7  57.0  38.1  38.0  36.3
     2      47.0  35.7  35.0  38.4  32.0  38.1

如果要获取一个人的所有检查信息，直接查询即可。<b>（注意：DataFrame上的基本索引是列索引）</b>

In [29]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      34.0  37.4
     2      42.0  35.2
2014 1      57.0  38.1
     2      35.0  38.4

In [ ]:
health_data.loc[2013]

## 4.4 多级索引的取值与切片

MultiIndex 的取值和切片操作很直观，可以直接把索引看成额外增加的维度。下面先介绍Series多级索引的取值与切片方法，再介绍DataFrame的用法。

### 4.4.1 Series多级索引

获取单个元素：

In [30]:
pop['California', 2000]

33871648

局部取值：

In [31]:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

局部切片，它要求MultiIndex是按顺序排列的：

In [32]:
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

如果索引已经排序，可以用较低层级的索引取值，第一层索引可以用空切片。

In [33]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

通过掩码选择数据：

In [34]:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

可以用列表选择数据：

In [35]:
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

### 4.4.2 DataFrame多级索引

由于DataFrame上的基本索引是列索引，所以前面多级索引的读取方法应用到 DataFrame 上就变成了对列操作。

In [36]:
health_data['Guido', 'HR']

year  visit
2013  1        34.0
      2        42.0
2014  1        57.0
      2        35.0
Name: (Guido, HR), dtype: float64

前面介绍的loc、iloc和ix索引器都可以使用，比如：

In [40]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      31.0  38.3
     2      45.0  36.2

在 loc 和 iloc 中还可以传递多个层级的索引元组：

In [41]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        31.0
      2        45.0
2014  1        40.0
      2        47.0
Name: (Bob, HR), dtype: float64

如果要获取所有人第一次到访时的心跳数据，直观的想法是用元组切片，比如：

In [42]:
health_data.loc[(:, 1), (:, 'HR')]

SyntaxError: invalid syntax (<ipython-input-42-fb34fa30ac09>, line 1)

但是 Pandas 不支持直接切片，应该使用 IndexSlice 对象来实现上述切片功能：

In [43]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,31.0,34.0,35.0
2014,1,40.0,57.0,38.0


可以看到，IndexSlice 对象的句法和元组切片的句法很像，只需要外层嵌套 IndexSlice 对象。

## 4.5 多级索引行列转换

在进行行列转换之前，一般要确保索引是有序的。如果 MultiIndex 不是有序的索引，那么大多数切片操作都会失败。

首先来看一个索引不按字典顺序排列的反例。

In [54]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1,2]])
data = pd.Series(np.random.rand(6), index)
data.index.names=['char', 'int']
data

char  int
a     1      0.178584
      2      0.054141
c     1      0.493602
      2      0.566581
b     1      0.394933
      2      0.532872
dtype: float64

In [ ]:
data['a':'b']

尽管从错误信息里面看不出具体的细节，但是问题是出在MultiIndex的无序排序上。<b>局部切片和许多其他相似操作都要求MultiIndex的各级索引是有序的。</b>
Pandas 提供的 sort_index() 和 sortlevel()方法都可以用来对索引进行排序。

In [56]:
data = data.sort_index()
data['a':'b']

char  int
a     1      0.178584
      2      0.054141
b     1      0.394933
      2      0.532872
dtype: float64

### 4.5.1 索引stack与unstack
多级索引还可以通过level参数设置转换的索引层级，比如，在前面的例子中，如果想把州名、年份作为列展开，那么对应的参数为：

In [79]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [69]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


使用 stack() 方法可以将 DataFrame 还原为 Series 对象。注意，这里年份自然变为一级索引，州变成了二级索引。

In [85]:
df=pop.unstack(level=0)
se=df.stack()
se

year  state     
2000  California    33871648
      New York      18976457
      Texas         20851820
2010  California    37253956
      New York      19378102
      Texas         25145561
dtype: int64

### 4.5.2 索引的设置与重置
DataFrame 还可以通过 set_index() 和 reset_index() 方法完成多级索引与数据列之间的转换。

In [87]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [91]:
pop_flat.set_index(['year','state'])

,,population
year,state,
2000,California,33871648
2010,California,37253956
2000,New York,18976457
2010,New York,19378102
2000,Texas,20851820
2010,Texas,25145561


In [92]:
pop_flat.set_index(['state','year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

### 4.5.3 多级索引的数据统计方法
Pandas自带许多数据统计的方法，比如 mean()、sum()和max()，对于层级索引，可以结合参数 level 实现对不同粒度数据的统计。

比如要计算每年各项指标的平均值，可以将参数level设置为索引year：

In [94]:
data_mean = health_data.mean(level='year')
data_mean

subject   Bob        Guido          Sue       
type       HR   Temp    HR   Temp    HR   Temp
year                                          
2013     38.0  37.25  38.0  36.30  31.5  37.45
2014     43.5  36.70  46.0  38.25  35.0  37.20

计算每个人（列）各项指标的平均值

In [102]:
data_mean = health_data.mean()
#这跟下面的语句是等价的
#data_mean = health_data.mean(axis=0)
data_mean

subject  type
Bob      HR      40.750
         Temp    36.975
Guido    HR      42.000
         Temp    37.275
Sue      HR      33.250
         Temp    37.325
dtype: float64

计算每年的各项指标的平均值：<b>（注意：聚合函数中，缺省轴axis=0为行，axis=1为列，这与DataFrame不同）</b>

先按 year 统计各人、各项指标的平均值，再按 项目 统计平均值

类似于SQL 中的 GroupBy 语句：group by year, type

In [113]:
data_mean = health_data.mean(level='year')
data_mean = data_mean.mean(axis=1,level='type')
data_mean

type,HR,Temp
year,,
2013,35.833333,37.000000
2014,41.500000,37.383333


## 4.6 Panel数据

Panel 和 Panel4D 可以分别看成是 Series 和 DataFrame 的三维和四维形式。多级索引在大多数情况下更实用、更直观。而且，<b>Panel采用密集数据存储形式，而多级索引采用稀疏数据存储形式</b>，这在存储高维度大数据时非常重要。

## 作业

1、使用 MultiIndex 创建一个具有行、列多级索引的 DataFrame。

2、实现查询、切片功能。